In [1]:
from pygraphblas import *
import pygraphblas.descriptor
import csv
import sys



In [11]:
#Load data from CSV format
class DataLoader:
    
    def __init__(self, path):
        self.path = path
        
    def load_node(self, filename):
        filename = self.path + filename
        with open(filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile, delimiter='|', quotechar='"')
            original_ids = [int(row['id']) for row in reader]
            id_mapping = {}
            for index in range(len(original_ids)):
                id_mapping[original_ids[index]] = index
            
        return original_ids, id_mapping

    def load_edge(self, filename, start_mapping, end_mapping, typ=BOOL, drop_dangling_edges=False):
        filename = self.path + filename
        with open(filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile, delimiter='|', quotechar='"')
            row_ids = []
            col_ids = []
            values = []
            for row in reader:
                start_id = int(row['id:START_ID'])
                end_id = int(row['id:END_ID'])
                if not drop_dangling_edges or (start_id in start_mapping and end_id in end_mapping):
                    row_ids.append(start_mapping[start_id])
                    col_ids.append(end_mapping[end_id])
                    values.append(1)
        
            edge_matrix = Matrix.from_lists(
            row_ids,
            col_ids,
            values,
            nrows=len(start_mapping), 
            ncols=len(end_mapping), 
            typ=typ)
            return edge_matrix

In [13]:
path = 'outputDir-1k/'
loader = DataLoader(path)
data_size = 2


vertices = {}
mapping = {}
matrices = {}
vertices['Comment'], mapping['Comment'] = loader.load_node('comment.csv')
vertices['Person'], mapping['Person'] = loader.load_node('person.csv')
matrices['replyOf'] = loader.load_edge('comment_replyOf_comment.csv', mapping['Comment'], mapping['Comment'])

